In [2]:
import numpy as np
%load_ext Cython

In [3]:
npz = np.load("UUU-2frag-4A.npz")
max_rmsd = 3 #overlapping cutoff (<3.0A recommended)
max_msd = max_rmsd**2
print(max_msd)
preatoms = np.load("UUU-e3-preatoms.npy")
postatoms = np.load("UUU-e3-postatoms.npy")
outnpz = "test"
connections = npz['interactions-0'].astype(np.int32)
nat = preatoms.shape[1] // 3
preatoms = preatoms.reshape(preatoms.shape[0], nat, 3)
postatoms = postatoms.reshape(postatoms.shape[0], nat, 3)

9


In [8]:
%%cython_pyximport subconnect
#%%cython --a
cimport numpy as np
cimport cython
import numpy as np
ctypedef int[2] two_int
@cython.boundscheck(False)
@cython.wraparound(False)  # turn off negative index wrapping for entire function
@cython.cdivision(True)
def subconnect(double[:,:,:] preatoms,
               double[:,:,:] postatoms,
               int[:,:] connections, float max_msd):
    cdef float r
    cdef int i, x, at, c0, c1, c, nat
    cdef double[:,:] coor1, coor2
    cdef int nconn = len(connections)
    cdef np.ndarray[char, ndim=1, mode="c"] new_connections = np.zeros(nconn, dtype=np.uint8)
    c = 0
    nat = preatoms.shape[1]
    for i in range(nconn):
        c0, c1 = connections[i,0],connections[i,1] 
        if c0 != c:
            c = c0
            coor1 = preatoms[c0]
        coor2 = postatoms[c1]
        r = 0
        for at in range(nat):
            for x in range(3):
                r += (coor2[at,x]-coor1[at,x])**2
        r = r/nat
        if r <= max_msd:
            new_connections[i] = 1
    return np.array(connections)[new_connections.astype(np.bool)]

In [9]:
%%timeit
new = subconnect(preatoms, postatoms, connections, max_msd)

859 µs ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
new

array([[ 19,  61],
       [ 19,  67],
       [ 37, 507],
       [ 42, 353],
       [ 72, 507],
       [ 77,   0],
       [ 84,  61],
       [ 84,  67],
       [ 95,   0],
       [ 95, 353],
       [101,  41],
       [101,  52],
       [101, 119],
       [101, 195],
       [103, 507],
       [110,   0],
       [110, 353],
       [118, 507],
       [121,  41],
       [121,  52],
       [132,  41],
       [132, 119],
       [132, 195],
       [137, 119],
       [137, 195],
       [143,  41],
       [143,  52],
       [143, 119],
       [143, 195],
       [143, 326],
       [144, 507],
       [150,  41],
       [150,  52],
       [150,  83],
       [154, 366],
       [164,  41],
       [164,  52],
       [164, 119],
       [164, 195],
       [164, 326],
       [179, 507],
       [183,  41],
       [183,  52],
       [183, 166],
       [189,  41],
       [189,  52],
       [202,  41],
       [202,  52],
       [202,  83],
       [209, 507],
       [253,   0],
       [253, 353],
       [278,

In [18]:
len(connections)

7152

In [19]:
len(new)

174

In [6]:
postatoms.dtype

dtype('float64')

In [8]:
connections.dtype

dtype('int64')